In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk
import torch
import string
import os
import tensorflow as tf
import time
from time import sleep
from string import punctuation
from sklearn import metrics
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TFBertModel
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet50 import ResNet50
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing import image

/Users/howardtangkulung/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
long_df = pd.read_csv('../raw_data/long_sypnosis/long_sypnosis.csv').drop(columns = "Unnamed: 0")

In [3]:
# resize all images to 350x350
# convert images to arrays (ignore corrupted images)
width, height = 256, 256
image_array = []
unidentified_count = 0
not_found_count = 0
for i in tqdm(range(long_df.shape[0])):
    try:
        image_path = f"../raw_data/posters/all/{long_df['imdb_id'][i]}.jpg"
        img = image.load_img(image_path, target_size=(width, height, 3))
        input_arr = image.img_to_array(img)
        image_array.append([long_df['imdb_id'][i], input_arr])
    except UnidentifiedImageError as e1:
        unidentified_count += 1
        pass
    except FileNotFoundError as e2:
        not_found_count += 1
        pass
print(f"{unidentified_count} files were unidentified and {not_found_count} files were not found")
img_array_df = pd.DataFrame(image_array, columns=["imdb_id", "image_array"])

100%|██████████| 4624/4624 [00:13<00:00, 336.69it/s]

294 files were unidentified and 386 files were not found


In [4]:
# join image_array_df and df on imdb_id (to delete imdb_id with corrupted images)
long_df = long_df.merge(img_array_df, on="imdb_id", how="right").drop(columns='image_url')
long_df = long_df.dropna().reset_index(drop=True)

In [6]:
long_df.head(1)

,movie,imdb_id,genre,sypnosis,image_array
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",An anime adaptation of the Hindu epic the Rama...,"[[[0.654902, 0.5686275, 0.40784314], [0.819607..."


# Preprocess Data

In [7]:
long_df["genre"] = (
    long_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )
long_df.head(1)

,movie,imdb_id,genre,sypnosis,image_array
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",An anime adaptation of the Hindu epic the Rama...,"[[[0.654902, 0.5686275, 0.40784314], [0.819607..."


In [8]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    return sentence

In [9]:
long_df['sypnosis'] = long_df['sypnosis'].apply(cleaning)
long_df.head(1)

,movie,imdb_id,genre,sypnosis,image_array
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",an anime adaptation of the hindu epic the rama...,"[[[0.654902, 0.5686275, 0.40784314], [0.819607..."


In [10]:
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()

def balanced_split(df, mlb, test_size=0.5):
    ind = np.expand_dims(np.arange(len(df)), axis=1)
    mlb.fit_transform(df["genre"])
    labels = mlb.transform(df["genre"])
    ind_train, _, ind_test, _ = iterative_train_test_split(
        ind, labels, test_size
    )
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [11]:
df_train, df_test = balanced_split(long_df, mlb, test_size=0.2)
# df_val, df_test = balanced_split(df_tmp, test_size=0.5)

In [12]:
df_val, df_test = balanced_split(df_test, mlb, test_size=0.5)

In [13]:
df_train.head(1)

,movie,imdb_id,genre,sypnosis,image_array
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",an anime adaptation of the hindu epic the rama...,"[[[0.654902, 0.5686275, 0.40784314], [0.819607..."


In [19]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['genre'])

#transform target variable
y_train = multilabel_binarizer.transform(df_train['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_train[f"{genre_names[i]}"] = y_train[:,i]

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['genre'])

#transform target variable
y_test = multilabel_binarizer.transform(df_test['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_test[f"{genre_names[i]}"] = y_test[:,i]

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_val['genre'])

#transform target variable
y_val = multilabel_binarizer.transform(df_val['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_val[f"{genre_names[i]}"] = y_val[:,i]

/var/folders/sy/4mvdh7yx2rn37237j7qbqrdr0000gn/T/ipykernel_51807/1415669347.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f"{genre_names[i]}"] = y_train[:,i]


In [21]:
print(y_test.shape, y_train.shape, y_val.shape)

(406, 21) (3131, 21) (407, 21)


In [24]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_text = df_train['sypnosis'].to_list()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_text = df_test['sypnosis'].to_list()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

val_text = df_val['sypnosis'].to_list()

In [25]:
train_encodings = tokenizer(train_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
test_encodings = tokenizer(test_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
val_encodings = tokenizer(val_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")

In [37]:
train_encodings['input_ids']

array([[  101,  2019,  8750, ...,  1996,  5698,   102],
       [  101,  2043,  1996, ...,  1517,  6506,   102],
       [  101,  2661, 17103, ...,  4154,  2332,   102],
       ...,
       [  101,  1996,  2048, ...,  6769,  1996,   102],
       [  101,  2019, 19104, ...,  2000,  5914,   102],
       [  101,  1996,  2149, ...,  2586,  2000,   102]])

In [40]:
train_imgs = df_train['image_array']
test_imgs = df_test['image_array']
val_imgs = df_val['image_array']

In [44]:
train_imgs[0].shape

(256, 256, 3)

In [45]:
# load the dataset and split into train/test sets
# REMEMBER TO DROP COLUMNS!!
X_train_img = train_imgs
X_train_text = train_encodings['input_ids']
y_train = y_train
X_test_img = test_imgs
X_test_text = test_encodings['input_ids']
y_test = y_test
X_val_img = val_imgs
X_val_text = val_encodings['input_ids']
y_val = y_val

In [53]:
print(X_train_img[0].shape, len(X_train_text[0]))

(256, 256, 3) 128


In [54]:
y_train.shape

(3131, 21)

# RESNET 50

In [55]:
def load_model():
    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    return model

In [56]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [57]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(512, activation='relu')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model

# BERT Model

In [58]:
# define the BERT-based text feature extractor
def build_text_model():
 bert_model = TFBertModel.from_pretrained('bert-base-uncased')
 inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
 outputs = bert_model(inputs)[1]
 text_model = tf.keras.Model(inputs=inputs, outputs=outputs)
 return text_model

# Multimodal Model

In [59]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
    model = load_model()
    model = set_nontrainable_layers(model)
    img_model = add_last_layers(model)
    text_model = build_text_model()
    img_input = tf.keras.layers.Input(shape=(256, 256, 3), name='img_input')
    text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_input')
    img_features = img_model(img_input)
    text_features = text_model(text_input)
    concat_features = tf.keras.layers.concatenate([img_features, text_features])
    x = tf.keras.layers.Dense(512, activation='relu')(concat_features)
    x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    multimodal_model = tf.keras.Model(inputs=[img_input, text_input], outputs=x)
    return multimodal_model

In [60]:
# build the multimodal model
num_classes = 21
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

94765736/94765736 [==============================] - 3s 0us/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 text_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 sequential (Sequential)        (None, 512)          89380724    ['img_input[0][0]']              
                                                                                                  
 model (Functional)             (None, 768)          109482240   ['text_input[0][0]']       

In [66]:
# compile the model and train on the train set
multimodal_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'binary_accuracy', 'categorical_accuracy'])

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = multimodal_model.fit([(X_train_img, X_train_text)], tf.convert_to_tensor(y_train), epochs=15, batch_size=32, validation_data=([(X_val_img, X_val_text)], tf.convert_to_tensor(y_val)), callbacks = [es])

ValueError: Data cardinality is ambiguous:
  x sizes: 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 2

In [67]:
X_train_img = X_train_img.tolist()
X_train_img

AttributeError: 'list' object has no attribute 'tolist'